# (1) Twitter Data
## (1.1) Getting Twitter data 2021 from API

In [1]:
import pandas as pd
from path import Path
from twarc import Twarc2, expansions
import json
import datetime
import os

In [2]:
from config import bearer_token

In [3]:
client = Twarc2(bearer_token=bearer_token)

In [4]:
user = 'elonmusk'
posts_dict = {
    'date':[],
    'text':[],
    'like_count':[],
    'reply_count':[],
    'retweet_count':[]
}

In [5]:
# pull posts from Twitter and create a dictionary
user_timeline = client.timeline(user=user, exclude_replies=True, start_time=datetime.datetime(2021,1,1, 0, 0, 0) )
for page in user_timeline:
    result = expansions.flatten(page)
    for tweet in result:
        posts_dict['date'].append(tweet['created_at'])
        posts_dict['text'].append(tweet['text'])
        posts_dict['like_count'].append(tweet['public_metrics']['like_count'])
        posts_dict['reply_count'].append(tweet['public_metrics']['reply_count'])
        posts_dict['retweet_count'].append(tweet['public_metrics']['retweet_count'])

In [6]:
# convert dictionary of posts to dataframe
twitter_2021 = pd.DataFrame.from_dict(posts_dict)
twitter_2021.head()

,date,text,like_count,reply_count,retweet_count
0,2021-07-21T19:55:51.000Z,RT @SpaceX: Dragon has autonomously re-docked ...,0,0,2897
1,2021-07-21T11:14:53.000Z,RT @NASA: LIVE: The @SpaceX Crew Dragon Endeav...,0,0,2193
2,2021-07-20T01:22:56.000Z,RT @SpaceX: First static fire test of Super He...,0,0,9240
3,2021-07-20T00:07:03.000Z,Full test duration firing of 3 Raptors on Supe...,120426,7759,6130
4,2021-07-19T17:44:50.000Z,"RT @inspiration4x: Generosity, Prosperity, Lea...",0,0,1932


In [7]:
twitter_2021['date'] = pd.to_datetime(twitter_2021['date']).dt.date.astype('datetime64')
twitter_2021.tail()

,date,text,like_count,reply_count,retweet_count
439,2021-01-07,This is called the domino effect https://t.co/...,366267,4496,37492
440,2021-01-04,"Because of the large footprint, it may seem fl...",57821,1386,1069
441,2021-01-04,Snow falling on Giga Berlin https://t.co/eTXMt...,148310,3642,6862
442,2021-01-02,So proud of the Tesla team for achieving this ...,109734,4145,6228
443,2021-01-02,"RT @Tesla: In 2020, we produced and delivered ...",0,0,6259


## (1.2) Getting Twitter data 2011 - 2020 from archive

In [8]:
# load twitter data from csv file
file_to_load = os.path.join('Data', 'elon_musk_tweets_2011-2020.csv')
twitter_archive = pd.read_csv(file_to_load)
twitter_archive.head()

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,0,1343644462036086785,1343320495127633920,1.609185e+12,2020-12-28 19:46:18,0,NaN,Entertainment will be critical when cars drive...,en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1,1343619610617077760,1343386617294295040,1.609179e+12,2020-12-28 18:07:33,0,NaN,@kimpaquette Just meeting with Larry Ellison t...,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'kimpaquette', 'name': 'Kim P...",NaN,NaN,NaN,NaN
2,2,1343608616960491521,1343576442722893825,1.609176e+12,2020-12-28 17:23:51,0,NaN,@richierichhhhh_ Absolutely,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'richierichhhhh_', 'name': 'R...",NaN,NaN,NaN,NaN
3,3,1343608530998153222,1343320495127633920,1.609176e+12,2020-12-28 17:23:31,0,NaN,What should Tesla do with in-car gaming in an ...,en,[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,4,1343431408052662273,1343043963096326147,1.609134e+12,2020-12-28 05:39:42,0,NaN,@PPathole @WSJ Absolutely,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'PPathole', 'name': 'Pranay P...",NaN,NaN,NaN,NaN


In [9]:
# select and rename columns
twitter_archive_clean = twitter_archive[['date', 'tweet', 'nlikes', 'nreplies', 'nretweets']]\
                            .loc[(twitter_archive['reply_to'] == '[]') & (twitter_archive['retweet'] == False)]
twitter_archive_clean.columns=['date', 'text', 'like_count', 'reply_count', 'retweet_count']

# convert date to datetime datatype
twitter_archive_clean['date'] = pd.to_datetime(twitter_archive_clean['date']).dt.date.astype('datetime64')

# drop last row with 1 tweet in 2010
twitter_archive_clean.drop(twitter_archive_clean.tail(1).index,inplace=True)

twitter_archive_clean.head()

,date,text,like_count,reply_count,retweet_count
0,2020-12-28,Entertainment will be critical when cars drive...,55085,2922,2611
3,2020-12-28,What should Tesla do with in-car gaming in an ...,33830,6932,884
6,2020-12-27,Try playing Polytopia in your Tesla! Great gam...,148037,5355,4186
34,2020-12-25,"Change your horn sound to 🐐, 🐍🎷, 💨 or holiday ...",187368,5373,6983
35,2020-12-25,Merry Christmas &amp; happy holidays! 🎁 https...,236833,7496,13288


## (1.3) Clean the twitter data

In [10]:
# concatenate 2 datasets to get tweets from 2011 to 2021
twitter_df_merged = pd.concat([twitter_2021, twitter_archive_clean])
twitter_df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4629 entries, 0 to 11715
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           4629 non-null   datetime64[ns]
 1   text           4629 non-null   object        
 2   like_count     4629 non-null   int64         
 3   reply_count    4629 non-null   int64         
 4   retweet_count  4629 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 217.0+ KB


In [11]:
# Drop the NaNs
twitter_df_merged.dropna()

,date,text,like_count,reply_count,retweet_count
0,2021-07-21,RT @SpaceX: Dragon has autonomously re-docked ...,0,0,2897
1,2021-07-21,RT @NASA: LIVE: The @SpaceX Crew Dragon Endeav...,0,0,2193
2,2021-07-20,RT @SpaceX: First static fire test of Super He...,0,0,9240
3,2021-07-20,Full test duration firing of 3 Raptors on Supe...,120426,7759,6130
4,2021-07-19,"RT @inspiration4x: Generosity, Prosperity, Lea...",0,0,1932
...,...,...,...,...,...
11711,2011-12-04,Am reading a great biography of Ben Franklin b...,65,17,9
11712,2011-12-03,That was a total non sequitur btw,53,31,6
11713,2011-12-03,"Great Voltaire quote, arguably better than Twa...",29,7,25
11714,2011-12-01,I made the volume on the Model S http://t.co/...,78,31,9


In [12]:
twitter_df_merged.to_csv('data/tweets_data_lda.csv', index=False)

In [13]:
# group tweets that posted at the same day
def f(x):
     return pd.Series(dict(like_count = x['like_count'].sum(),
                        reply_count = x['reply_count'].sum(),
                        retweet_count = x['retweet_count'].sum(),
                        text = "{%s}" % ', '.join(x['text'])))

In [14]:
twitter_df_merged = twitter_df_merged.groupby('date').apply(f).reset_index()
twitter_df_merged.head()


,date,like_count,reply_count,retweet_count,text
0,2011-12-01,267,63,24,{I made the volume on the Model S http://t.co...
1,2011-12-03,82,38,31,"{That was a total non sequitur btw, Great Volt..."
2,2011-12-04,65,17,9,{Am reading a great biography of Ben Franklin ...
3,2011-12-21,1330,87,597,{Yum! Even better than deep fried butter: htt...
4,2011-12-22,1349,132,206,{Model S options are out! Performance in red a...


In [15]:
twitter_df_merged.count()

date             1724
like_count       1724
reply_count      1724
retweet_count    1724
text             1724
dtype: int64

In [16]:
twitter_df_merged.to_csv('data/tweets_data_lda.csv', index=False)

## (1.4) Preprocessing the Twitter data

**Preprocess the data by making it all lowercase. Remove a reasonable set of stopwords from the dataset and tokenize. Then, report the 10 most common words and their count. We need to iterate this process, adding some stop words as we understand the structure of the data. Justify additional stop words we've added.**

In [17]:
# Data Pre-processing and make the tweets all lowercase and remove stopwords.
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from datetime import datetime
from nltk.stem import PorterStemmer
import re
import nltk
nltk.download('stopwords')

twitter_df = twitter_df_merged.copy()

# lower the tweets
twitter_df['preprocessed_text'] = twitter_df['text'].str.lower()

# remove apostrophe from words and url
twitter_df['preprocessed_text'] = [re.sub("('[a-z]+)\s", " ", row) for row in twitter_df['preprocessed_text']]
twitter_df['preprocessed_text'] = [re.sub("(')\s", " ", row) for row in twitter_df['preprocessed_text']]
twitter_df['preprocessed_text'] = [re.sub("(?:https:\/\/\S+)\s", "", row) for row in twitter_df['preprocessed_text']]

                                      
# filter out rest URLs
url_re = '(?:https?:\/\/)?(?:[^?\/\s]+[?\/])(.*)'
twitter_df['preprocessed_text'] = twitter_df['preprocessed_text'].apply(lambda row: ' '.join([word for word in row.split() if (not re.match(url_re, word))]))

# tokenize the tweets
tokenizer = RegexpTokenizer('[a-zA-Z]\w+\'?\w*')
twitter_df['tokenized_text'] = twitter_df['preprocessed_text'].apply(lambda row: tokenizer.tokenize(row))

#create an object of class PorterStemmer
porter = PorterStemmer()

# apply stemming
twitter_df['preprocessed_text'] = [porter.stem(row) for row in twitter_df['preprocessed_text']]   

# filter out stop words
en_stop_words = nltk.corpus.stopwords.words('english')
additional_stop_words =['amp', 'rt', 'th','co', 're', 've', 'kim', 'daca', 'us', 'it', 'th', 'you', 'haha', 'st', 'et', 'so', 'iii', 'also', 've', 'la', 're', 'the', 'https', 'wow', 'actually', 'due', 'ft', 'pcr', 'via', 'am', 'gt', 'com', 'since', 'in', 'me', 'and', 'btw', 'yesterday', 'ii', 'inu', 'on', 'http', 'to', 'vs', 'rd', 'ur', 'of', 'bs', 'km', 'est', 'em', 'lz', 'kms', 'aft', 'nd',  'here’s', 're', 'mqxfakpzf' 'mph', 'ht', 'etc', 'dm', 'doo']
en_stop_words.extend(additional_stop_words)

twitter_df['tokenized_text'] = twitter_df['tokenized_text'].apply(lambda row: [word for word in row if (not word in en_stop_words)])

df_tweets_clean = twitter_df.copy()
df_tweets_clean.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zkirsan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,date,like_count,reply_count,retweet_count,text,preprocessed_text,tokenized_text
0,2011-12-01,267,63,24,{I made the volume on the Model S http://t.co...,{i made the volume on the model s go to 11. no...,"[made, volume, model, go, need, work, miniatur..."
1,2011-12-03,82,38,31,"{That was a total non sequitur btw, Great Volt...","{that was a total non sequitur btw, great volt...","[total, non, sequitur, great, voltaire, quote,..."
2,2011-12-04,65,17,9,{Am reading a great biography of Ben Franklin ...,{am reading a great biography of ben franklin ...,"[reading, great, biography, ben, franklin, isa..."
3,2011-12-21,1330,87,597,{Yum! Even better than deep fried butter: htt...,{yum! even better than deep fried butter: yeah...,"[yum, even, better, deep, fried, butter, yeah,..."
4,2011-12-22,1349,132,206,{Model S options are out! Performance in red a...,{model s options are out! performance in red a...,"[model, options, performance, red, black, deli..."


In [18]:
df_tweets_clean = df_tweets_clean[['date', 'text', 'tokenized_text', 'like_count', 'reply_count', 'retweet_count']]
df_tweets_clean.head(10)

,date,text,tokenized_text,like_count,reply_count,retweet_count
0,2011-12-01,{I made the volume on the Model S http://t.co...,"[made, volume, model, go, need, work, miniatur...",267,63,24
1,2011-12-03,"{That was a total non sequitur btw, Great Volt...","[total, non, sequitur, great, voltaire, quote,...",82,38,31
2,2011-12-04,{Am reading a great biography of Ben Franklin ...,"[reading, great, biography, ben, franklin, isa...",65,17,9
3,2011-12-21,{Yum! Even better than deep fried butter: htt...,"[yum, even, better, deep, fried, butter, yeah,...",1330,87,597
4,2011-12-22,{Model S options are out! Performance in red a...,"[model, options, performance, red, black, deli...",1349,132,206
5,2011-12-24,{The Russians are having some challenges with ...,"[russians, challenges, rockets, many, engineer...",117113,1370,8434
6,2011-12-26,{Walked around a neighborhood recently rebuilt...,"[walked, around, neighborhood, recently, rebui...",558,102,171
7,2011-12-27,{If you ever wanted to know the *real* truth a...,"[ever, wanted, know, real, truth, moon, landin...",39,13,34
8,2011-12-28,{@TheOnion So true :)},"[theonion, true]",12,7,1
9,2011-12-29,{Am not saying that is *necessarily* good or b...,"[saying, necessarily, good, bad, reality, forc...",187,39,41


In [19]:
# the most common words and their count
def get_most_freq_words(str, n=None):
    vect = CountVectorizer().fit(str)
    bag_of_words = vect.transform(str)
    sum_words = bag_of_words.sum(axis=0) 
    freq = [(word, sum_words[0, idx]) for word, idx in vect.vocabulary_.items()]
    freq =sorted(freq, key = lambda x: x[1], reverse=True)
    return freq[:n]
  
len(get_most_freq_words([ word for tweet in df_tweets_clean.tokenized_text for word in tweet]))

8197

In [20]:
df_tweets_clean.to_csv('data/tweets_data_2010_2020.csv', index=False)

## (1.5) Upload dataset to SQL Database

In [21]:
import sqlalchemy
from sqlalchemy import create_engine, inspect
import psycopg2
from config import user, password, hostname

In [22]:
# Create engine
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{hostname}/twitter_vs_stocks')

# Use the Inspector to explore the database
inspector = inspect(engine)

In [23]:
df_tweets_clean.to_sql('tweets_text', engine, if_exists ='append',method='multi', index=False)

# (2 ) Stock data

## (2.1) Getting the stock data

In [34]:
from yahoo_fin.stock_info import get_data

In [36]:
# historical daily data from Yahoo finance
tesla_df = get_data("tsla", start_date = '2011-01-01', end_date = None, index_as_date = False, interval="1d")
tesla_df

,date,open,high,low,close,adjclose,volume,ticker
0,2011-01-03,5.368000,5.400000,5.180000,5.324000,5.324000,6415000,TSLA
1,2011-01-04,5.332000,5.390000,5.204000,5.334000,5.334000,5937000,TSLA
2,2011-01-05,5.296000,5.380000,5.238000,5.366000,5.366000,7233500,TSLA
3,2011-01-06,5.366000,5.600000,5.362000,5.576000,5.576000,10306000,TSLA
4,2011-01-07,5.600000,5.716000,5.580000,5.648000,5.648000,11239500,TSLA
...,...,...,...,...,...,...,...,...
2651,2021-07-16,654.679993,656.700012,642.200012,644.219971,644.219971,16339800,TSLA
2652,2021-07-19,629.890015,647.200012,621.289978,646.219971,646.219971,21297100,TSLA
2653,2021-07-20,651.989990,662.390015,640.500000,660.500000,660.500000,15442700,TSLA
2654,2021-07-21,659.609985,664.859985,650.289978,655.289978,655.289978,13910800,TSLA


## (2.2) Clean the stock data

In [37]:
# Drop adjclose column
tesla_df = tesla_df.drop(columns=["adjclose", "ticker"])
tesla_df.head()

,date,open,high,low,close,volume
0,2011-01-03,5.368,5.400,5.180,5.324,6415000
1,2011-01-04,5.332,5.390,5.204,5.334,5937000
2,2011-01-05,5.296,5.380,5.238,5.366,7233500
3,2011-01-06,5.366,5.600,5.362,5.576,10306000
4,2011-01-07,5.600,5.716,5.580,5.648,11239500


In [38]:
# Determine data types for each column
tesla_df.dtypes

date      datetime64[ns]
open             float64
high             float64
low              float64
close            float64
volume             int64
dtype: object

## (2.3) Preprocessing the Stock Data

In [39]:
# Calculate change in stock price
tesla_df['change'] = tesla_df['close'].diff()
tesla_df.head(10)

,date,open,high,low,close,volume,change
0,2011-01-03,5.368,5.400,5.180,5.324,6415000,NaN
1,2011-01-04,5.332,5.390,5.204,5.334,5937000,0.010
2,2011-01-05,5.296,5.380,5.238,5.366,7233500,0.032
3,2011-01-06,5.366,5.600,5.362,5.576,10306000,0.210
4,2011-01-07,5.600,5.716,5.580,5.648,11239500,0.072
5,2011-01-10,5.634,5.736,5.610,5.690,6713500,0.042
6,2011-01-11,5.718,5.742,5.384,5.392,8551000,-0.298
7,2011-01-12,5.402,5.480,5.304,5.392,4822000,0.000
8,2011-01-13,5.392,5.394,5.232,5.244,3618000,-0.148
9,2011-01-14,5.230,5.316,5.122,5.150,5960000,-0.094


In [40]:
tesla_df.to_csv('data/tesla_stocks.csv', index=False)

## (2.4) Upload dataset to SQL Database

In [42]:
tesla_df.to_sql('stock', engine, if_exists ='append',method='multi', index=False)